In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
from matplotlib import cm

In [11]:
#Load in the data
X_old = pd.read_csv("../Data/trainX.csv")
y_old = pd.read_csv("../Data/trainY.csv")
target_val_old = pd.read_csv("../Data/testX.csv")

X = pd.read_csv("../Data/trainX-test.csv")
y = pd.read_csv("../Data/trainY-test.csv")
target_val = pd.read_csv("../Data/testX-test.csv")

['belongs_to_collection',
 'has_homepage',
 'originally_english',
 'topStudio',
 'usa_produced',
 'released_in_english',
 'has_tagline',
 'has_keywords',
 'topLeadActor',
 'topLeadDirector',
 'topLeadExecProd',
 'topLeadProducer',
 'topLeadComposer',
 'topLeadDirectorPhoto',
 'topLeadEditor',
 'log_budget_processed',
 'log_genre_rank',
 'log_num_genres',
 'log_overview_len',
 'log_numTopStudios',
 'log_num_studios',
 'log_num_production_countries',
 'log_day_of_week',
 'log_year',
 'log_month',
 'log_week_of_year',
 'log_season',
 'log_runtime_processed',
 'log_num_languages',
 'log_title_len',
 'log_num_cast',
 'log_numTopActors',
 'log_numTopDirectors',
 'log_numTopExecProd',
 'log_numTopProducers',
 'log_numTopComposers',
 'log_numTopDirectorsPhoto',
 'log_numTopEditors']

In [12]:
target_val_old.log_budget_processed.describe()

count    4398.000000
mean       16.348816
std         1.800308
min         0.693147
25%        16.012735
50%        16.588099
75%        17.147715
max        19.376192
Name: log_budget_processed, dtype: float64

In [13]:
X.columns

Index(['belongs_to_collection', 'has_homepage', 'originally_english',
       'topStudio', 'usa_produced', 'released_in_english', 'has_tagline',
       'has_keywords', 'topLeadActor', 'topLeadDirector', 'topLeadExecProd',
       'topLeadProducer', 'topLeadComposer', 'topLeadDirectorPhoto',
       'topLeadEditor', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s',
       'log_budget_processed', 'log_genre_rank', 'log_num_genres',
       'log_overview_len', 'log_numTopStudios', 'log_num_studios',
       'log_studioRank', 'log_num_production_countries', 'log_day_of_week',
       'log_year', 'log_month', 'log_week_of_year', 'log_season',
       'log_runtime_processed', 'log_num_languages', 'log_title_len',
       'log_num_cast', 'log_numTopActors', 'log_actorRanks', 'log_num_crew',
       'log_num_male_crew', 'log_num_female_crew', 'log_numTopDirectors',
       'log_directorsRank', 'log_numTopExecProd', 'log_execProdRank',
       'log_numTopProducers', 'log_producersRank', 'log_numTopCom

In [14]:
X_old.columns

Index(['belongs_to_collection', 'has_homepage', 'originally_english',
       'topStudio', 'usa_produced', 'released_in_english', 'has_tagline',
       'has_keywords', 'topLeadActor', 'topLeadDirector', 'topLeadExecProd',
       'topLeadProducer', 'topLeadComposer', 'topLeadDirectorPhoto',
       'topLeadEditor', 'log_budget_processed', 'log_genre_rank',
       'log_num_genres', 'log_overview_len', 'log_numTopStudios',
       'log_num_studios', 'log_num_production_countries', 'log_day_of_week',
       'log_year', 'log_month', 'log_week_of_year', 'log_season',
       'log_runtime_processed', 'log_num_languages', 'log_title_len',
       'log_num_cast', 'log_numTopActors', 'log_numTopDirectors',
       'log_numTopExecProd', 'log_numTopProducers', 'log_numTopComposers',
       'log_numTopDirectorsPhoto', 'log_numTopEditors', 'log_num_crew',
       'log_num_male_crew', 'log_num_female_crew'],
      dtype='object')

In [15]:
X.log_budget_processed.describe()

count    2999.000000
mean       16.186409
std         1.952961
min         0.693147
25%        15.424949
50%        16.636890
75%        17.448669
max        20.427616
Name: log_budget_processed, dtype: float64

In [17]:
#X_new.log_budget_processed.describe()

In [18]:
#X.log_budget_processed = X_new.log_budget_processed
#target_val.log_budget_processed = target_val_new.log_budget_processed


In [19]:
y = y['log_revenue']

In [20]:
# create a 75/25 stratified split of the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7, test_size=0.20)

In [21]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (2399, 57) (2399,)
Test set: (600, 57) (600,)


In [22]:
y_test

1306    11.881962
2036    11.775297
568     16.692910
1896    17.815544
2497    14.645964
          ...    
2108    13.127866
2004    16.738203
1124    19.828196
2401    18.952889
905     18.346211
Name: log_revenue, Length: 600, dtype: float64

In [23]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [24]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [25]:
import math
def RMSLE(predict, target):
    total = 0 
    for k in range(len(predict)):
        LPred= np.log1p(predict[k]+1)
        LTarg = np.log1p(target[k] + 1)
        if not (math.isnan(LPred)) and  not (math.isnan(LTarg)): 
            total = total + ((LPred-LTarg) **2)
        
    total = total / len(predict)        
    return np.sqrt(total)

## Random Forest

In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns

In [27]:
rfr_base_model = RandomForestRegressor(random_state = 42)
rfr_base_model.fit(X_train, y_train.values.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [28]:
y_pred = rfr_base_model.predict(X_test)

In [29]:
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
rmse

1.675119754557742

In [30]:
# Calculate the absolute errors
train_errors = abs(rfr_base_model.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(y_pred - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

Train Set Accuracy: 96.15 %
Test Set Accuracy: 90.86 %


In [31]:
RMSLE(np.expm1(y_pred), np.expm1(y_test.values))

1.6740143031630637

In [32]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 25],
    'max_features': ['auto', 'log2', .6],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [5, 6, 8],
    'n_estimators': [100, 200, 400]
}
# Create a based model
rfr = RandomForestRegressor()
# Instantiate the grid search model
rfr, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, rfr, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-rfr.best_score_))
print(rfr.best_params_)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 20.5min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed: 25.0min finished


1.7237927304174283
{'bootstrap': True, 'max_depth': 25, 'max_features': 0.6, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 400}


In [33]:
print('Base Model:')
base_accuracy = evaluate(rfr_base_model, X_test, y_test.values)
print()
print('Model after Tuning:')
rfr_best_model = rfr.best_estimator_
best_accuracy = evaluate(rfr_best_model, X_test, y_test.values)

print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))

Base Model:
Average Error: 1.1649 degrees.
Accuracy = 90.86%.

Model after Tuning:
Average Error: 1.1632 degrees.
Accuracy = 90.46%.
Improvement of -0.44%.


In [34]:
feature_importances = rfr_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))

Feature ranking:
1. log_budget_processed (0.265666)
2. log_budget_to_year_ratio (0.245830)
3. log_num_male_crew (0.047625)
4. log_studioRank (0.039987)
5. log_runtime_to_year_ratio (0.030931)
6. log_num_studios (0.029925)
7. log_actorRanks (0.028847)
8. log_num_cast (0.023541)
9. topStudio (0.023228)
10. log_overview_len (0.020187)
11. log_genre_rank (0.019879)
12. log_year (0.019290)
13. log_runtime_processed (0.018567)
14. log_num_crew (0.018030)
15. log_week_of_year (0.015308)
16. log_numTopStudios (0.015126)
17. log_title_len (0.011973)
18. log_editorsRank (0.011699)
19. log_producersRank (0.009623)
20. log_day_of_week (0.009384)
21. belongs_to_collection (0.007366)
22. log_num_female_crew (0.007215)
23. log_month (0.006903)
24. log_directorsPhotoRank (0.006420)
25. log_num_genres (0.005966)
26. log_directorsRank (0.005166)
27. log_season (0.005021)
28. log_composersRank (0.004501)
29. log_num_production_countries (0.004169)
30. has_homepage (0.003760)
31. 2010s (0.003242)
32. has_

In [ ]:
from sklearn.metrics import mean_squared_error,make_scorer
from hyperopt import tpe,hp,Trials
from hyperopt.fmin import fmin

In [ ]:
seed=2
def objective(params):
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    msl=int(params['min_samples_leaf'])
    mss=int(params['min_samples_split'])
    model=RandomForestRegressor(n_estimators=est,max_depth=md,min_samples_leaf=msl,min_samples_split=mss)
    model.fit(X_train,y_train.values.ravel())
    pred=model.predict(X_test)
    score=mean_squared_error(y_test.values,pred)
    return score

def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators',100,250),
           'max_depth':hp.uniform('max_depth',5,100),
           'min_samples_leaf':hp.uniform('min_samples_leaf',1,10),
           'min_samples_split':hp.uniform('min_samples_split',2,8)}
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=100,rstate=np.random.RandomState(seed))
    return best

trial=Trials()
best=optimize(trial)

In [ ]:
print(best)

In [ ]:
rfr_opt=RandomForestRegressor(max_depth =38, min_samples_leaf= 3, min_samples_split = 8, n_estimators = 148)
rfr_opt.fit(X_train,y_train.values.ravel())
pred_rfr_opt=rfr_opt.predict(X_test)
score_rfr_opt=mean_squared_error(y_test.values,pred_rfr_opt)
print(np.sqrt(score_rfr_opt))

In [ ]:
# Calculate the absolute errors
train_errors = abs(rfr_opt.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(pred_rfr_opt - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

## Extra Trees

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std

In [ ]:
et_base_model = ExtraTreesRegressor()
et_base_model.fit(X_train, y_train.values.ravel());

In [ ]:
y_pred = et_base_model.predict(X_test)

In [ ]:

# Calculate the absolute errors
errors = abs(y_pred - y_test.values)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test.values)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
et_model = ExtraTreesRegressor()
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

et_model, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, et_model, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-et_model.best_score_))
print(et_model.best_params_)

In [ ]:
base_model = ExtraTreesRegressor()
base_model.fit(X_train, y_train.values.ravel())
base_accuracy = evaluate(base_model, X_test, y_test.values)

et_best_model = et_model.best_estimator_
best_accuracy = evaluate(et_best_model, X_test, y_test.values)


print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))

Model Performance
Average Error: 2.8057 degrees.
Accuracy = 73.40%.
Model Performance
Average Error: 2.5961 degrees.
Accuracy = 74.66%.
Improvement of 1.71%.

In [ ]:
feature_importances = et_best_model.feature_importances_

In [ ]:
feature_importances = et_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))


In [ ]:
seed=2
def objective(params):
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    msl=int(params['min_samples_leaf'])
    mss=int(params['min_samples_split'])
    model=ExtraTreesRegressor(n_estimators=est,max_depth=md,min_samples_leaf=msl,min_samples_split=mss)
    model.fit(X_train,y_train.values.ravel())
    pred=model.predict(X_test)
    score=mean_squared_error(y_test.values,pred)
    return score

def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators',100,250),
           'max_depth':hp.uniform('max_depth',5,100),
           'min_samples_leaf':hp.uniform('min_samples_leaf',1,10),
           'min_samples_split':hp.uniform('min_samples_split',2,8)}
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=100,rstate=np.random.RandomState(seed))
    return best

trial=Trials()
best=optimize(trial)

In [ ]:
print(best)

In [ ]:
et_opt=ExtraTreesRegressor(max_depth =97, min_samples_leaf= 5, min_samples_split = 2, n_estimators = 197)
et_opt.fit(X_train,y_train.values.ravel())
pred_et_opt=rfr_opt.predict(X_test)
score_et_opt=mean_squared_error(y_test.values,pred_rfr_opt)
print(np.sqrt(score_rfr_opt))

In [ ]:
# Calculate the absolute errors
train_errors = abs(et_opt.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(pred_et_opt - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

In [ ]:
RMSLE(np.expm1(pred_et_opt), np.expm1(y_test.values))

## XGBoost

In [ ]:
from numpy import loadtxt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
xgb_base_model = xgb.XGBRegressor()
xgb_base_model.fit(X_train, y_train.values.ravel())

In [ ]:
y_pred = xgb_base_model.predict(X_test)

In [ ]:
# Calculate the absolute errors
train_errors = abs(xgb_base_model.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(y_pred - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

In [ ]:
xgb_model = xgb.XGBRegressor()
param_grid = {
    'n_estimators': [100, 200, 400],
    'colsample_bytree': [1.0, 1.1, 1.5],
    'max_depth': [1, 5, 100],
    'reg_alpha': [1.2, 1.3, 1.4],
    'reg_lambda': [ 1.2, 1.3],
    'subsample': [0.8, 0.9, 1.0]
}

xgb_model, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, xgb_model, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-xgb_model.best_score_))
print(xgb_model.best_params_)

In [ ]:
print('Base Model:')
base_accuracy = evaluate(xgb_base_model, X_test, y_test.values)
print()
print('Model after Tuning:')
xgb_best_model = xgb_model.best_estimator_
best_accuracy = evaluate(xgb_best_model, X_test, y_test.values)


print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))


In [ ]:
feature_importances = xgb_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))

## LightGBM

In [ ]:
# lightgbm for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot

# evaluate the model
lgbm_base_model = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
lgbm_base_model = LGBMRegressor()
lgbm_base_model.fit(X_train, y_train)
# make a single prediction
row = [[2.02220122, 0.31563495, 0.82797464, -0.30620401, 0.16003707, -1.44411381, 0.87616892, -0.50446586, 0.23009474, 0.76201118]]
yhat = lgbm_base_model.predict(X_test)
print('Prediction: %.3f' % yhat[0])

In [ ]:
mse = mean_squared_error(y_test,yhat)
rmse = np.sqrt(mse)
rmse

In [ ]:
# Calculate the absolute errors
errors = abs(yhat - y_test.values)
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test.values)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%')

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [100, 150],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8],
    'max_depth': [1, 2, 5],
    'reg_alpha': [1.0, 1.1, 1.2],
    'reg_lambda': [1.2, 1.3],
    'subsample': [0.3,0.35, 0.4]

}
# Create a based model
lgbm = LGBMRegressor()
lgbm, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, lgbm, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-lgbm.best_score_))
print(lgbm.best_params_)

In [ ]:
lgbm_base_model = LGBMRegressor()
lgbm_base_model.fit(X_train, y_train.values.ravel())
base_accuracy = evaluate(lgbm_base_model, X_test, y_test.values)

lgbm_best_model = lgbm.best_estimator_
best_accuracy = evaluate(lgbm_best_model, X_test, y_test.values)


print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))

In [ ]:
feature_importances = lgbm_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))

## Evaluation

In [ ]:
results = pd.DataFrame(columns = ['id','revenue'])

In [ ]:
preds=et_opt.predict(target_val)

In [ ]:
for i in range(3001, len(target_val) + 3001):
    print(i)

In [ ]:
#results['id'] = range(3001, len(target_val) + 3001)
#results['revenue'] = np.expm1(preds)
#results

In [ ]:
#results.to_csv("ET_baseline2-85p.csv",index=False)